In [2]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import os

In [3]:
def Histograms(image, title):
    
    hist = cv2.calcHist(image, [0], None, [255], [0, 255])
    plt.ylabel('Pixels')
    plt.xlabel('Darkness')
    plt.title(title)
    plt.plot(hist)
    plt.xlim([0,256])
    plt.savefig(title + 'histogram.png')
    plt.clf()

In [4]:
images = {}
for x in os.listdir("Images"):
    images[x] = cv2.imread("Images/" + x)

In [5]:
class Window:
    
    def __init__(self, image, n, p):
        
        self.xlimit = image.shape[1]+n
        self.ylimit = image.shape[0]+n
        self.topleft = (0,0)
        self.bottomright = (n,n)
        self.prev = n
        self.height = n
        self.pace = (p, p)
        
        try:
            self.channels = image.shape[2]
        except:
            self.channels = 1
            
    def Position(self):
        
        return self.topleft, self.bottomright
    
    def NextPosition(self):
        
        #New Line Movement
        if(self.bottomright + self.pace)[0] >= (self.xlimit - self.height):
            return (0, self.topleft[1] + self.pace[1]), (self.height, self.bottomright[1] + self.pace[1])
        
        #Moving one pixel to the Right
        else:
            return (self.topleft[0] + self.pace[0], self.topleft[1]), (self.bottomright[0] + self.pace[0], self.bottomright[1])
        
    def NextMove(self):
        
        #Move Position of Window
        self.topleft, self.bottomright = self.NextPosition()
        return self.topleft, self.bottomright
    
    def InBounds(self, templeft = None, tempright = None):
        
        if templeft is None:
            templeft = self.topleft
        if tempright is None:
            tempright = self.bottomright 
            
        return tempright[0] <= self.xlimit and tempright[1] <= self.ylimit and templeft[0] >= 0 and templeft[1] >= 0
    
    def newY(self):
        
        if self.prev == self.bottomright[1]:
            return False
        
        else:
            
            self.prev = self.bottomright[1]
            return True
        
    def getImgBounds(self, image):
        
        img = []
        
        for i in range(self.topleft[1], self.bottomright[1]):
            
            if i >= image.shape[0]:
                continue
                
            img.append(image[i][self.topleft[0]: self.bottomright[0]])
            
        if self.channels == 1:
            
            return np.resize(np.array(img), (self.height, self.height)) #self.height twice since square
        
        else:
            
            return np.resize(np.array(img), (self.height, self.height, self.channels))

In [6]:
#Sliding Window Operations
show = False

window = Window(images["dababy.jpg"], 100, 50)
topleft, bottomright = window.Position()

image = cv2.rectangle(images["dababy.jpg"].copy(), topleft, bottomright, (0, 0, 255))
cv2.imwrite("Output/dababy.png", image, [cv2.IMWRITE_PNG_COMPRESSION, 0])

tempright, templeft = window.NextPosition()

while window.InBounds(bottomright):
    image = cv2.rectangle(images["dababy.jpg"].copy(), topleft, bottomright, (0, 0, 255))
    if show:
        cv2.imshow("Window", image)
        cv2.waitKey(int(1/35*1000))
        
    topleft, bottomright = window.NextMove()
    tempright = window.NextPosition()
    
cv2.destroyAllWindows()

In [7]:
class Kernel:
    
    def __init__(self, kernel, weight):
        
        self.kernel = kernel
        self.weight = weight 
        
    def Filter(self, roi, axis = 0, channels = 1):
        
        output = []
        
        if axis == 2:
            
            for i in range(channels):
                
                if channels == 1:
                    _filter = self.kernel * roi
                
                else:
                    _filter = self.kernel * roi[:, :, i]
                    
                filsum = _filter.sum()
                
                if channels == 1:
                    _filter = self.kernel.T * roi
                
                else:
                    
                    _filter = self.kernel.T * roi[:, :, i]
                    
                filsum2 = _filter.sum()
                
                output.append((((filsum**2) + (filsum2 **2)) ** (1/2)) * self.weight)
                
            return np.array(output)
        
        else:
            
            if axis == 0:
                kernel = self.kernel 
            else:
                self.kernel.T
                
            for i in range(channels):
                
                if channels == 1:
                    _filter = kernel * roi
                
                else:
                    _filter = kernel * roi[:, :, i]
                    
                output.append(_filter.sum() * self.weight)
                
            return np.array(output)
        
        
    def FilterImage(self, image, pace = 1, window = None, axis = 0):
        
        img = []
        line = []
        
        if window is None:
            
            move = Window(image, self.kernel.shape[0], pace)
            
        else:
            
            image = window.getImgBounds(image)
            move = Window(image, self.kernel.shape[0], pace)
            
        topleft, _ = move.NextPosition()
        while move.InBounds(topleft):
            
            roi = move.getImgBounds(image)
            
            if move.newY():
                img.append(line)
                line = []
                
            line.append(self.Filter(roi, axis, move.channels))
            
            move.NextMove()
            topleft, _ = move.NextPosition()
            
        return np.array(img)
        
   

In [8]:
class Sobel:
    
    def __init__(self, weight):
        
        self.kernel = Kernel(np.array([[-1 , 0, 1],
                                       [-2, 0 , 2],
                                       [-1, 0 , 1]]), weight)
        
    def FilterImage(self, image, pace = 1, window = None, axis = 2):
        return self.kernel.FilterImage(image, pace, window, axis)
    
class Gaussian:
    
    def __init__(self, size, weight):
        
        gauss = size // 2
        
        x = np.arange(0, size, 1, float)
        
        y = x[:, np.newaxis]
        
        tempx = tempy = size // 2
        self.kernel = Kernel(np.exp(-4 * np.log(2) * ((x - tempx) ** 2 + (y - tempy) ** 2) / gauss ** 2), weight)
        
    def FilterImage(self, image, pace = 1, window = None, axis = 0):
        return self.kernel.FilterImage(image, pace, window, axis)
    
class Bilinear:
    
    def __init__(self, weight):
        
        self.kernel = Kernel(np.array([[1, 2, 1],
                                       [2, 4, 2],
                                       [1, 2, 1]]), weight)
        
    def FilterImage(self, image, pace = 1, window = None, axis = 0):
        return self.kernel.FilterImage(image, pace, window, axis)

In [9]:
sobel = Sobel(1)

In [10]:
for image in images:
    win = Window(images[image], 300, 1)
    roi = window.getImgBounds(images[image])
    
    cv2.imwrite("Output/RoI/" + image + "BeforeFilter.jpg", roi, [cv2.IMWRITE_PNG_COMPRESSION, 0])
    fil = sobel.FilterImage(image = images[image], window = win, axis = 2)
    cv2.imwrite("Output/RoI/" + image + "AfterFilter.jpg", fil, [cv2.IMWRITE_PNG_COMPRESSION, 0])

In [11]:
image = images["dababy.jpg"].copy()
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
win = Window(image, 300, 1)
roi = win.getImgBounds(image)

cv2.imwrite("Output/RoI/Dababy_BeforeFilter.jpg", roi, [cv2.IMWRITE_PNG_COMPRESSION, 0])
fil = sobel.FilterImage(image = image, window = win)
cv2.imwrite("Output/RoI/Dababy_AfterFilter.jpg", fil, [cv2.IMWRITE_PNG_COMPRESSION, 0])

Histograms(cv2.imread("Output/RoI/Dababy_AfterFilter.jpg"), "After Filter Histogram")

<Figure size 432x288 with 0 Axes>

In [12]:
for image in images:
    fil = sobel.FilterImage(image = images[image], axis = 2)
    cv2.imwrite("Output/RoI/Sobel2/" + image + "AfterFilter.jpg", fil, [cv2.IMWRITE_PNG_COMPRESSION, 0])

In [13]:
bilinear = Bilinear(1/8)
gaussian = Gaussian(5, 1/4)

filters = {"Sobel": sobel,
          "Bilinear": bilinear,
           "Gaussian": gaussian}

In [14]:
for image in images:
    for fil in filters:
        if fil == "Sobel":
            filt = filters[fil].FilterImage(image=images[image], axis = 0)
            cv2.imwrite("Output/SobelX/" + image + "AfterFilter.jpg", filt,  [cv2.IMWRITE_PNG_COMPRESSION, 0])
            filt = filters[fil].FilterImage(image=images[image], axis = 0)
            cv2.imwrite("Output/SobelY/" + image + "AfterFilter.jpg", filt,  [cv2.IMWRITE_PNG_COMPRESSION, 0])
            
        else:
            filt = filters[fil].FilterImage(image = images[image])
            cv2.imwrite("Output/" + fil + "/" + image + "AfterFilter.jpg", filt, [cv2.IMWRITE_PNG_COMPRESSION, 0])